<a href="https://colab.research.google.com/github/changsin/MIU_ML/blob/main/notebooks/09.decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree

The following is an explanation of Decision Tree (section 8.4, pp. 200~) in Ertel's Artificial Intelligence. The data used is reproduced below. The last column _Skiing_ represents the target decision of either going or not going to skiing.


| Day | Snow_Dist | Weekend | Sun | Skiing |
| ----| --------|-----------|------| ------|
| 1 | $ \leq $100 | yes | yes | yes |
| 2 | $ \leq $100 | yes | yes | yes |
| 3 | $ \leq $100 | yes | no | yes |
| 4 | $ \leq $100 | no | yes | yes |
| 5 | >100 | yes | yes | yes |
| 6 | >100 | yes | yes | yes |
| 7 | >100 | yes | yes | no |
| 8 | >100 | yes | no | no |
| 9 | >100 | no | yes | no |
| 10 | >100 | no | yes | no |
| 11 | >100 | no | no | no |

## Entropy

The entropy is calculated according to Shannon's formula:

$ H(p) = H(p_1;...p_n) = -\Sigma_{n=1}^{n}p_i log_2 p_i = H(D) $

Applied to the skiing example in the text book, we calculate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
p1 = 5/11
p2 = 6/11
p3 = 7/11

probs = np.array([p1, p2])

def calculate_entropy(probs):
  entropy = 0
  for p in probs:
    if p != 0:
      entropy += p*np.log2(p)
  
  return -entropy



H = -(p1*np.log2(p1)) - (p2*np.log2(p2))
H

calculate_entropy(probs)

0.9940302114769565

In [ ]:
calculate_entropy(np.array([2/7, 5/7]))

0.863120568566631

In [ ]:
calculate_entropy(np.array([4/4, 0/4]))

-0.0

In [ ]:
calculate_entropy(np.array([6/11, 5/11]))

0.9940302114769565

## Information Gain

For dataset D, the information gained is thus:

$$ I(D) = 1 - H(D) $$

The information gain through attribute A is defined as:
$$ G(D, A) = \Sigma_{i=1}^{n}\frac{|D_i|}{|D|}I(D_i) - I(D) $$

or if we rewrite it in terms of entropy:

$$ = H(D) - \Sigma_{i=1}^{n}\frac{|D_i|}{|D|}H(D_i) $$

Applied to our skiing example, for the choice of snow distribution, we get:

In [ ]:
D = 11
D_snow_little = 4
D_snow_big = 7
H_D = calculate_entropy(np.array([6/11, 5/11]))
H_snow_little = calculate_entropy(np.array([4/4, 0/4]))
H_snow_big = calculate_entropy(np.array([2/7, 5/7]))
H_snow_big

0.863120568566631

0.863 is the entropy we get when we branch the decision tree for snow distribution. 

In [ ]:
def calculate_information_gain(dataset_count, initial_entropy, subs):
  """
    dataset_count: total number of entries in the dataset
    initial_entropy: the initial entropy of the dataset
    subs: a list of positive and negative counts for each sub-branch:
      e.g., [[4, 0], [2, 5]] means that for snow_dist,
       for the positive case, <= 100 give you 4 correct and 0 incorrect answers.
       for the negative case, 2 incorrect and 5 correct answers.
  """
  sub_entropy = 0

  for sub in subs:
    sub_counts = np.array(sub)
    sub_total = np.sum(sub_counts)
    sub_probs = np.array([ di/sub_total for di in sub_counts])

    entropy = calculate_entropy(sub_probs)
    print("subentropy", entropy)

    sub_entropy += (sub_total/dataset_count)*entropy

  return initial_entropy - sub_entropy

# How to build a decision tree with information gain metrics

Using the information gain formula, we can now calculate and compare what is the best way to build a decision tree. Given there are three attributes, we calculate the information gain for each as follows.



### A1: Snow distribution

In [ ]:
H_D = calculate_entropy(np.array([6/11, 5/11]))
sub_snow = [[4, 0],
            [2, 5]]
calculate_information_gain(11, H_D, sub_snow)

subentropy -0.0
subentropy 0.863120568566631


0.44477166784364586

### A2: Weekend

In [ ]:
sub_weekend = [[5, 2],
               [1, 3]]
calculate_information_gain(11, H_D, sub_weekend)

subentropy 0.863120568566631
subentropy 0.8112781244591328


0.14976144076759756

### A3: Sun

In [ ]:
sub_sun = [[5, 3],
            [1, 2]]
calculate_information_gain(11, H_D, sub_sun)

subentropy 0.9544340029249649
subentropy 0.9182958340544896


0.0494520727893939

Based on the results, we know that snow distribution is the first branch question we should ask to get the most information gain.